# Match Tactical Report

This notebook creates a single-match tactical report from processed StatsBomb event data. It:

- loads `data_processed/events.csv`,
- generates tactical visuals,
- saves figures to `reports/figures/`,
- writes a short narrative with three insights in plain language.


In [ ]:
from pathlib import Path
import sys

import pandas as pd
from IPython.display import Markdown, display

ROOT = Path.cwd()
if not (ROOT / 'src').exists():
    ROOT = ROOT.parent

sys.path.append(str(ROOT))

from src.tactical_metrics import (
    passes_into_final_third,
    progressive_passes,
    box_entries,
    defensive_actions_in_opposition_half,
    possessions,
)
from src.viz import (
    plot_team_heatmap,
    plot_passing_network,
    plot_shot_map_xg,
    plot_defensive_action_map,
)


In [ ]:
events_path = ROOT / 'data_processed' / 'events.csv'
fig_dir = ROOT / 'reports' / 'figures'
fig_dir.mkdir(parents=True, exist_ok=True)

if not events_path.exists():
    raise FileNotFoundError(f'Missing processed data file: {events_path}')

events = pd.read_csv(events_path)
events['match_id'] = pd.to_numeric(events['match_id'], errors='coerce')
events['team_id'] = pd.to_numeric(events['team_id'], errors='coerce')

match_ids = sorted(events['match_id'].dropna().unique())
if not match_ids:
    raise ValueError('No match_id values found in processed events.')

match_id = int(match_ids[0])
match_events = events[events['match_id'] == match_id].copy()

team_counts = match_events['team_id'].value_counts(dropna=True)
if team_counts.empty:
    raise ValueError('No team_id values found for selected match.')

focus_team_id = int(team_counts.index[0])

print(f'Selected match_id: {match_id}')
print(f'Focus team_id: {focus_team_id}')
print(f'Events in match: {len(match_events):,}')


In [ ]:
# Build tactical data slices
match_possessions = possessions(match_events)
match_final_third = passes_into_final_third(match_events)
match_progressive = progressive_passes(match_events)
match_box_entries = box_entries(match_events)
match_def_actions = defensive_actions_in_opposition_half(match_events)

passes_only = match_events[match_events['type_name'] == 'Pass'].copy()
shots_only = match_events[match_events['type_name'] == 'Shot'].copy()

print('Possessions:', len(match_possessions))
print('Final third passes:', len(match_final_third))
print('Progressive passes:', len(match_progressive))
print('Box entries:', len(match_box_entries))
print('Defensive actions in opposition half:', len(match_def_actions))


In [ ]:
# Save four tactical visuals
heatmap_path = fig_dir / f'match_{match_id}_team_{focus_team_id}_heatmap.png'
network_path = fig_dir / f'match_{match_id}_team_{focus_team_id}_passing_network.png'
shotmap_path = fig_dir / f'match_{match_id}_team_{focus_team_id}_shot_map_xg.png'
defmap_path = fig_dir / f'match_{match_id}_team_{focus_team_id}_def_actions.png'

plot_team_heatmap(match_events, team_id=focus_team_id, output_path=heatmap_path)
plot_passing_network(passes_only, team_id=focus_team_id, output_path=network_path)
plot_shot_map_xg(shots_only, team_id=focus_team_id, output_path=shotmap_path)
plot_defensive_action_map(match_def_actions, team_id=focus_team_id, output_path=defmap_path)

print('Saved:')
print('-', heatmap_path)
print('-', network_path)
print('-', shotmap_path)
print('-', defmap_path)


In [ ]:
# Plain-language tactical narrative (3 insights)
team_possessions = match_possessions[match_possessions['team_id'] == focus_team_id].copy()
team_prog = match_progressive[match_progressive['team_id'] == focus_team_id].copy()
team_f3 = match_final_third[match_final_third['team_id'] == focus_team_id].copy()
team_box = match_box_entries[match_box_entries['team_id'] == focus_team_id].copy()
team_def = match_def_actions[match_def_actions['team_id'] == focus_team_id].copy()

avg_pos_len = float(team_possessions['event_count'].mean()) if not team_possessions.empty else 0.0
avg_pos_dur = float(team_possessions['duration_seconds'].mean()) if not team_possessions.empty else 0.0

insight_md = f'''
### Tactical Insights (Match {match_id}, Team {focus_team_id})

1. **Ball progression profile**: The team produced **{len(team_prog)} progressive passes** and **{len(team_f3)} passes into the final third**.
A progressive pass means the ball is moved forward by a meaningful distance, which matters because it shows how often the team breaks lines instead of circulating safely.

2. **Penalty-area access**: The team recorded **{len(team_box)} box entries** (passes or carries into the penalty area).
This matters because entries into the box usually create higher-quality chance situations than shots from distance.

3. **Defensive positioning**: The team made **{len(team_def)} defensive actions in the opposition half**.
This matters because winning the ball high up the pitch can reduce the distance to goal and create faster attacks after regains.

Supplementary context: average possession length was **{avg_pos_len:.1f} events** over **{avg_pos_dur:.1f} seconds**.
'''

display(Markdown(insight_md))
